### Making the Series Table
This table will have the series_id, series metadata, and county_id for all series within every county

In [1]:
import pandas as pd
import numpy as np
import time
import requests
import json

# Function to rename a feature that has the county's name in it
def get_series_name(county_name, series_title):
    ndx = series_title.find(county_name)
    return series_title[:ndx]

In [2]:
# Load the list of all counties
df_county_table = pd.read_csv('county_table.csv')
print(df_county_table.shape)
print(df_county_table.head())
counties_list = df_county_table['county_id'].values
counties_names = df_county_table['name'].values

# Initialize the series table
df_series_table = pd.DataFrame()
series_table_cols = ['id', 'title', 'observation_start', 'observation_end', 'frequency', 'units', 'seasonal_adjustment']

(3240, 3)
   county_id                name  state_id
0      27336  Autauga County, AL     27335
1      27337  Baldwin County, AL     27335
2      27338  Barbour County, AL     27335
3      27339     Bibb County, AL     27335
4      27340   Blount County, AL     27335


In [3]:
# This cell and the next cell is where the series table starts to be filled
# There are a lot of series so I ran these two cells many times back and forth
#  (I wanted to save the data rather than store it in temporary memory)
df_series_table = pd.read_csv('series_table_3100.csv')
print(df_series_table.shape)
print(df_series_table.head())

(300663, 8)
  frequency               id observation_end observation_start  \
0    Annual  2020RATIO001001      2018-01-01        2010-01-01   
1   Monthly    ACTLISCOU1001      2020-03-01        2016-07-01   
2   Monthly  ACTLISCOUMM1001      2020-03-01        2017-07-01   
3   Monthly  ACTLISCOUYY1001      2020-03-01        2017-07-01   
4   Monthly       ALAUTA1LFN      2020-02-01        1990-01-01   

       seasonal_adjustment                                              title  \
0  Not Seasonally Adjusted                                  Income Inequality   
1  Not Seasonally Adjusted            Housing Inventory: Active Listing Count   
2  Not Seasonally Adjusted  Housing Inventory: Active Listing Count Month-...   
3  Not Seasonally Adjusted  Housing Inventory: Active Listing Count Year-O...   
4  Not Seasonally Adjusted                               Civilian Labor Force   

     units  county_id  
0    Ratio      27336  
1    Level      27336  
2  Percent      27336  
3  Perce

In [4]:
start = 3100
for county_id, county_name in zip(counties_list[start:], counties_names[start:]):
    time.sleep(.5)
    params = {
        'category_id': county_id,
        'api_key': 'e76fcf746d3ca3cc025c0803dd212fc8',
        'file_type': 'json'
    }
    r = requests.get(url = 'https://api.stlouisfed.org/fred/category/series', params=params)
    res = r.json()
    seriess = res['seriess']
    series_in_county = pd.DataFrame()
    for series in seriess:
        temp = { your_key: series[your_key] for your_key in series_table_cols }
        temp['title'] = get_series_name(county_name, temp['title'])
        series_in_county = series_in_county.append(temp, ignore_index=True)
    series_in_county['county_id'] = county_id
    df_series_table = df_series_table.append(series_in_county)

print(df_series_table.shape)

(314211, 8)


In [5]:
df_series_table.to_csv('series_table_all.csv', index=False)